# $\mathscr I$ space
Space that models the information packet associated with a protein

In [40]:
class Information:
    def __init__(self, tupla):
        self.tupla = tupla
        
    def print():
        print(self.tupla)
        
    def vectorize():
        return np.array(self.tupla)   

# $\mathscr M$ set
OOP description of the set of possible mutations

In [41]:
class Mutation:
    def __init__(self, descriptor):
        self.descriptor = descriptor

# $\mathscr P$ space
OOP description of the space of proteins
<center><h1><b>Content (functions acting on $\mathscr P$)</b></h1></center>
<ul>
    <li><code>compute_q_value</code> : $\mathscr P \rightarrow \mathbb R_+$</li>
    computes the <i>q value</i> of the protein, performing the simulation. The result is then stored into the variable         <code>q_value</code>
    <li><code>mutate</code> : $\mathscr P \times \mathscr M \rightarrow \mathscr P$</li>
    produce a new protein, starting from the current one and the mutation requested
    <li><code>information_neighbors</code> : $\mathscr p \mapsto \{ (\mathscr q,I_{\mathscr q}) \}_{\mathscr q \in \mathcal N(\mathscr p)}$ </li>
    returns a mapping that associates each neighbor on the <i>protein lattice</i> with an information packet in the set $\mathscr I$
</ul>

# V2: using BuildModel for mutations

In [42]:
import os
import pandas as pd
import subprocess
class Protein:
    instances = 0
    
    def __init__(self, filename):
        self.filename = filename
        self.q_value  = None                               # unitialized q value
        # set of available_mutations
    def build_neighbors_on_mutation_lattice(self, threshold): # build the lattices based on the ALA threshold's Gibbs free energy
        # generate config file with possible mutations for the current protein
        os.system("python3 mutation_generator.py " + self.filename + " " + str(threshold))
        # Protein.instances += 1 ## questo sarebbe da gestire in un altro modo

        # mutate the protein 
        print("Generating  mutated pdbs...")
        msg = subprocess.run(["foldx_20231231 --command=BuildModel --pdb=" + (self.filename) + " --mutant-file=individual_list.txt > mutation_output.txt"], shell = True)
 
        print(msg)
        return self

    def build_information_tables(self):
        os.system("mv Dif_*.fxout mutation.csv") # use the csv format instead of the one of foldx
        X = pd.read_csv("mutation.csv", sep='\t',skiprows=8).to_numpy()
        # information packet
        I = X[:,2:21]

        # mutations
        M = X[:,0] + ".pdb" # add the suffix for the pdb type, used for the simulation name
        
        I = I.astype(float)
        return M,I
    
    def compute_q_value(self):      # note: this is a batch, run always this scripts as a whole    
        print("-----------------------------------Protein %s         -----------------------------------" %self.filename[:-4])
        print("Ionizing the structure for the simulation...")
        subprocess.run(["python3 protein_ionization.py " + self.filename], shell=True, stdout = subprocess.DEVNULL)
        
        print("Running the simulation...")
        subprocess.run(["python3 launch_simulation.py " + self.filename ], shell=True, stdout = subprocess.DEVNULL)
        print("Computing the Q-value...")
        proc = subprocess.run(["python3 evaluate_qvalue.py " + self.filename ], shell=True, stdout = subprocess.PIPE)
        #print(proc.stdout)
        self.q_value = float(proc.stdout)
        print(self.q_value)

# Learner
Agent able to learn from experience

In [43]:
import numpy as np
class Learner:
    def __init__(self, data_collection = 2):
        self.X               = np.array([])
        self.Y               = np.array([])
        self.data_collection = data_collection
    def learn(self,x,y):
        if len(self.X) == 0:
            # first entry
            self.X = (x * 1)[None,:]
            self.Y = y * 1
        else:
            self.X = np.r_[self.X,x[None,:]]
            self.Y = np.r_[self.Y,y]
            self.X = self.X[-100:]
            self.Y = self.Y[-100:]
        #print("I know %d things" % self.X.shape[0] )
    def predict(self,x):
        # prediction function...
        if len(self.X) < self.data_collection:                    # if i have not enough data
            print("Learner : sto sparando a caso")
            return np.random.randn(len(x))
        else:
                print("Learner : sto ragionando")

                print("input = " ,x.shape)
                print("X.shape\t",self.X.shape)
                print("Y.shape\t",self.Y.shape)
            
                k       = lambda x,y : (1 + x@y)**2 #np.exp( - (x - y)@(x - y) )
            
                K_train = np.array([
                    [
                            k(a,b)
                        for b in self.X
                    ]
                    for a in self.X
                ])
                K_in    = np.array([
                    [
                            k(a,b)
                        for b in self.X
                    ]
                    for a in x
                ])
                return K_in @ np.linalg.solve( K_train + 1e-3 * np.eye(len(K_train)), self.Y)

# Guide
Agent who drives the configuration exploration, modulating the trust wrt the Learner

In [44]:
class Guide:
    def __init__(self):
        self.eta = 5.
        
    def trust(self):
        print("Guida dice : Costruisco fiducia nel Learner")
        if self.eta < 50:
            self.eta *= 1.1          # if I build trust, the "confidence in the learner" increase
        
    def untrust(self):
        print("Guida dice : Perdo fiducia nel Learner")
        self.eta *= 1./1.1          # if I disappoint you, otherwise...

        
    def softmax(self,probabilityDistribution):
        # returns the homotopied probability distribution
        
        return np.exp(probabilityDistribution)** self.eta / (np.exp(probabilityDistribution) ** self.eta).sum()

    def choose(self,probabilityDistribution):
        # takes as input a probability distribution and "homotopies" it with the uniform distribution
        # according to its trust.
        # returns the extracted index
        normalize               = lambda x : (x - x.min())/(x.max() - x.min())
        probabilityDistribution = 1 - normalize(probabilityDistribution)
        P = self.softmax(probabilityDistribution)
        return np.random.choice(len(P),p = P)

# Team
Class that models the cooperative system between the learner and the guide

In [45]:
class Team:
    def __init__(self):
        self.G = Guide()
        self.L = Learner()
        
    def optimize(self,protein):
        protein.compute_q_value()                                                # compute the q value of the input protein
        for i in range(100):                                                     # loop
            information_neighbors    =  protein.information_neighbors()          # map each neighbour with an information packet
            probability_distribution =  self.L.predict(information_neighbors)    # returns a dictionary that associate each mutation with a probability distribution
            selected_mutation        =  self.G.choose(probability_distribution)  # select the next mutation according to the learner's opinion
            
            new_protein              =  protein.mutate(selected_mutation)        # performs the mutation
            new_protein.compute_q_value()                                        # compute the q value for the current mutation
            
            self.L.learn(                                                        # give to the Learner the knowledge about 
                    information_neighbors[selected_mutation],                    # the mapping between chosen mutation information packet
                    new_protein.get_q_value()                                    # and the q value associated to the chosen mutation
            )
            
            if( new_protein.get_q_value() <= protein.get_q_value() ):            # greedy descent
                protein = new_protein                                            # since q value has been computed for new_protein it's stored in new_protein and therefore at this point in the protein
                self.G.trust()                                                   # builds trust between guide and learner
            else:
                self.G.untrust()                                                 # damages trust between guide and learner
            print( protein.get_q_value() )

# main

In [46]:
def main_loop(debug_version = False):
    P   =  P_0  = Protein("s.pdb")         # genero l oggetto proteina
    print(P.filename[:-4])
    print("calculating q value of starting protein...")
    if not debug_version:
        #P.compute_q_value() # commment this to avoid  a simulation at the beginning
        P.q_value = 402
    else:
        P.q_value = 402
    
    print("%s ha q value pari a %s " % (P.filename, P.q_value))

    L           = Learner()
    G           = Guide()
    # genero il csv che descrive i pacchetti di informazine associati ai vicini sul lattice delle permutazioni
    # costruisco le tabelle a partire dal csv appena generato
    epochs = 30
    proteina_nuova = True
    #thresholds = np.linspace(1.5, 1., int((1.5 - 1.) / 0.05 + 1))
    for i in range(epochs):
        if proteina_nuova:
            print("Obtaining neighbours info...")
            if not debug_version:
                P.build_neighbors_on_mutation_lattice(1)
            else:
                0 # mutations.csv è gia presente
            proteina_nuova = False
        M,I = P.build_information_tables()
        scores = L.predict(I)
        choice = G.choose(scores)
        selected_mutation             = '.'.join(M[choice].split(".")[:2])
        print("la mutazione selezionata è %s" % selected_mutation)
        selected_mutation_information = I[choice]
        P_new = Protein(selected_mutation)
        old_qval = P.q_value 
        print("epoch %d" % i )
        if not debug_version:
            P_new.compute_q_value()
            delta_q = np.abs(old_qval - P_new.q_value)
        else:
            # sparo a caso un q value
            P_new.q_value = P.q_value + int( (np.random.uniform() * 2 - 1.) * 50 )
        if P_new.q_value <= P.q_value:
            proteina_nuova = True
            if P_new.q_value < P.q_value:
                G.trust()
            P = P_new
        else:
            G.untrust()
        # I let the learner lear about the association
        L.learn(selected_mutation_information, P_new.q_value)
        
        print("%s ha q value pari a %s " % (P_new.filename, P_new.q_value))
    return P_new
    
P_new = main_loop()

s
calculating q value of starting protein...
s.pdb ha q value pari a 402 
Obtaining neighbours info...
   ********************************************
   ***                                      ***
   ***             FoldX 5 (c)              ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

Start FoldX AlaScan >>>

1 models read: s.pdb
s
Your file run OK
End time of FoldX: Mon Nov 27 11:06:08 2023
Total time spend: 1.06093 seconds.
Total mutations (reduced for thi

/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


Running the simulation...
Computing the Q-value...


/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


399.0
Guida dice : Costruisco fiducia nel Learner
s_26.pdb ha q value pari a 399.0 
Obtaining neighbours info...
   ********************************************
   ***                                      ***
   ***             FoldX 5 (c)              ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

Start FoldX AlaScan >>>

1 models read: s_26.pdb
s_26
Your file run OK
End time of FoldX: Mon Nov 27 11:10:22 2023
Total time spend: 1.0756 seconds.
Total mutations (

...

396.0
Guida dice : Perdo fiducia nel Learner
s_26_23_19_11.pdb ha q value pari a 396.0 
Learner : sto ragionando
input =  (19, 19)
X.shape	 (29, 19)
Y.shape	 (29,)
la mutazione selezionata è s_26_23_19_9.pdb
epoch 29
-----------------------------------Protein s_26_23_19_9         -----------------------------------
Ionizing the structure for the simulation...


mv: cannot stat 'Dif_*.fxout': No such file or directory
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)
/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


Running the simulation...
Computing the Q-value...


/usr/local/lib/vmd/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /usr/local/lib/vmd/vmd_LINUXAMD64)


390.0
Guida dice : Perdo fiducia nel Learner
s_26_23_19_9.pdb ha q value pari a 390.0 


In [6]:
import numpy as np
np.linspace(1.5, 1., int((1.5 - 1.) / 0.05 + 1))

array([1.5 , 1.45, 1.4 , 1.35, 1.3 , 1.25, 1.2 , 1.15, 1.1 , 1.05, 1.  ])